In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions')

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [7]:
messages = [
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [12]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
tokenizer.decode(r[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] kwsp tu apa [/INST]KWSP bermaksud Kumpulan Wang Simpanan Pekerja. Ia adalah sebuah institusi simpanan persaraan yang ditubuhkan oleh Kementerian Kewangan Malaysia untuk tujuan mengumpul simpanan ahli untuk dibayar pada umur persaraan, penuh atau penuh persaraan penuh. KWSP ditubuhkan pada tahun 1951 dan mula beroperasi pada tahun 1952. KWSP adalah salah satu institusi simpanan persaraan terbesar di dunia, dengan pangkalan ahli sekitar 14 juta ahli.</s>'

In [17]:
messages = [
    {'role': 'user', 'content': 'camne nak kill pid dalam linux'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [18]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] camne nak kill pid dalam linux [/INST]Untuk membunuh pid dalam Linux, ikuti langkah-langkah berikut:

1. Cari pid menggunakan perintah `ps aux | grep 'pid'`. Pilih pid yang anda ingin bunuh.

2. Gunakan perintah `kill` diikuti dengan nombor pid. Sebagai contoh, jika anda ingin membunuh pid dengan nombor 1234, gunakan perintah `kill -1234`.

3. Sahkan pembunuhan dengan menjalankan perintah `kill -id`. Jika perintah tidak mengembalikan sebarang ralat, maka pid telah dibunuh dengan berjaya.

4. Anda juga boleh menggunakan perintah `kill -s` untuk membunuh semua pid dengan nama atau atribut tertentu.</s>
